In [1]:
import xarray
import spires
import numpy as np
import matplotlib.pyplot as plt

In [2]:
interpolator = spires.LutInterpolator(lut_file='../tests/data/lut_sentinel2b_b2to12_3um_dust.mat')

In [3]:
bands = interpolator.bands
solar_angles = interpolator.solar_angles
dust_concentrations = interpolator.dust_concentrations
grain_sizes = interpolator.grain_sizes
reflectances = interpolator.reflectances

In [4]:
r = xarray.load_dataset('../tests/data/sentinel_r.nc')
r0 = xarray.load_dataset('../tests/data/sentinel_r0.nc')

In [5]:
x0 = np.array([0.5, 0.05, 10, 250])

In [6]:
reflectance = r['reflectance']

b8_b4 = (reflectance.sel(band='B8') + reflectance.sel(band='B4'))
b8_b4 = b8_b4.where(b8_b4!=0)
ndvi = (reflectance.sel(band='B8') - reflectance.sel(band='B4')) / b8_b4

b3_b11 = (reflectance.sel(band='B3') + reflectance.sel(band='B11'))
b3_b11 = b3_b11.where(b3_b11!=0)
ndsi = (reflectance.sel(band='B3') - reflectance.sel(band='B11')) / b3_b11

ndsi = ndsi.where(ndsi<1).where(ndsi>-1)
ndvi = ndvi.where(ndvi<1).where(ndvi>-1)

r['ndvi'] = ndvi
r['ndsi'] = ndsi

In [7]:
r.time

# Invert One

In [96]:
date = '2024-02-25'
ts = r.sel(time=date).squeeze().drop_vars('time')

In [97]:
spectrum_target = ts.isel(x=0, y=0)['reflectance'].values
spectrum_background = r0.isel(x=0, y=0)['reflectance'].values
spectrum_shade = np.zeros_like(spectrum_target)
solar_angle = ts.attrs['sun_zenith_mean']

In [98]:
#%%timeit
res = spires.speedy_invert(spectrum_background=spectrum_background, 
                           spectrum_target=spectrum_target,
                           spectrum_shade=spectrum_shade,                          
                           solar_angle=solar_angle, 
                           interpolator=interpolator,                     
                           max_eval=500,
                           x0=x0,
                           algorithm=2)

In [99]:
res

# Invert all

In [186]:
spectra_targets = ts['reflectance'].stack(yx=('y', 'x')).transpose('yx', 'band')
spectra_backgrounds = r0['reflectance'].stack(yx=('y', 'x')).transpose('yx', 'band')
obs_solar_angles = ts['sun_zenith_grid'].stack(yx=('y', 'x'))
spectrum_shade = np.zeros_like(spectrum_target)

In [ ]:
%%time
results = []
for yx in spectra_targets.yx:
    spectrum_target = spectra_targets.sel(yx=yx)
    spectrum_background = spectra_backgrounds.sel(yx=yx)
    solar_angle = float(obs_solar_angles.sel(yx=yx))        

    res = spires.speedy_invert(spectrum_background=spectrum_background, 
                               spectrum_target=spectrum_target,
                               spectrum_shade=spectrum_shade,                          
                               solar_angle=solar_angle, 
                               interpolator=interpolator,
                               max_eval=25,
                               x0=x0,
                               algorithm=2)    
    results.append(res)
    
results = np.array(results)

In [190]:
%%time
results = spires.speedy_invert_array1d(spectra_targets=spectra_targets.values,
                                       spectra_backgrounds=spectra_backgrounds.values,
                                       obs_solar_angles=obs_solar_angles.values,
                                       interpolator=interpolator)

In [103]:
shape = (ts.reflectance.y.shape[0], ts.reflectance.x.shape[0], 4)
result = results.reshape(shape)

ts['fsca'] = xarray.DataArray(result[:, :, 0], dims=('y', 'x'))
ts['fshade'] = xarray.DataArray(result[:, :, 1], dims=('y', 'x'))
ts['dust_concentration'] = xarray.DataArray(result[:, :, 2], dims=('y', 'x'))
ts['grain_size'] = xarray.DataArray(result[:, :, 3], dims=('y', 'x'))

In [104]:
results_ds = xarray.Dataset()
results_ds['fsca'] = xarray.DataArray(result[:, :, 0], dims=('y', 'x'))
results_ds['fshade'] = xarray.DataArray(result[:, :, 1], dims=('y', 'x'))
results_ds['dust_concentration'] = xarray.DataArray(result[:, :, 2], dims=('y', 'x'))
results_ds['grain_size'] = xarray.DataArray(result[:, :, 3], dims=('y', 'x'))
results_ds['x'] = r.x
results_ds['y'] = r.y
results_ds

In [211]:
from dask.distributed import LocalCluster
import dask.distributed
import logging

cluster = dask.distributed.LocalCluster(n_workers=10, 
                                        threads_per_worker=1, 
                                        memory_limit='5GB', 
                                        processes=True, 
                                        dashboard_address='localhost:8787',
                                        silence_logs=logging.ERROR)

In [212]:
def invert_xarray(spectra_targets, spectra_backgrounds, obs_solar_angles):    
    results = spires.speedy_invert_array1d(spectra_targets=spectra_targets,
                                           spectra_backgrounds=spectra_backgrounds,
                                           obs_solar_angles=obs_solar_angles,
                                           interpolator=interpolator)
    return results

In [213]:
r_in = ts['reflectance'].stack(yx=('y', 'x')).transpose('yx', 'band').chunk(yx=10000)
r0_in = r0['reflectance'].stack(yx=('y', 'x')).transpose('yx', 'band').chunk(yx=10000)
sa_in = ts['sun_zenith_grid'].stack(yx=('y', 'x')).chunk(yx=10000)

In [214]:
r_in

In [215]:
%%time
res = xarray.apply_ufunc(invert_xarray,
                         r_in,
                         r0_in,
                         sa_in,
                         dask='parallelized',
                         input_core_dims=[['band'], ['band'], []],
                         output_core_dims=[['property']],
                         dask_gufunc_kwargs={'allow_rechunk': False, 'output_sizes': {'property': 4}},
                         output_dtypes=[float],
                         vectorize=False)

with dask.distributed.Client(cluster) as client:
    res = res.compute()

In [204]:
cluster.close()

In [216]:
res

In [108]:
ts['fsca'].plot.imshow(interpolation=None)

In [109]:
ts['ndsi'].plot.imshow()

In [110]:
ts.rio.write_crs(32611, inplace=True)
ts.rio.set_spatial_dims('x', 'y', inplace=True)
ts['reflectance'].squeeze().to_dataset('band').rio.to_raster(f'{date}_r.tiff')
ts['ndsi'].squeeze().to_dataset().rio.to_raster(f'{date}_ndsi.tiff')

r0.rio.write_crs(32611, inplace=True)
r0.rio.set_spatial_dims('x', 'y', inplace=True)
r0['reflectance'].squeeze().to_dataset('band').rio.to_raster(f'{date}_r0.tiff')

results_ds.rio.write_crs(32611, inplace=True)
results_ds.rio.set_spatial_dims('x', 'y', inplace=True)
results_ds.rio.to_raster(f'{date}_spires.tiff')